In [1]:
import pickle
import os
import time
import numpy as np
import polychrom

from polychrom import polymerutils
from polychrom import forces
from polychrom import forcekits
from polychrom.simulation import Simulation
from polychrom.starting_conformations import grow_cubic
from polychrom.hdf5_format import HDF5Reporter, list_URIs, load_URI, load_hdf5_file

import simtk.openmm 
import os 
import shutil


import pyximport; pyximport.install()
from smcTranslocator import smcTranslocatorDirectional

import warnings
import h5py 
import glob

## Defining simulation parameters

In [2]:

# -------defining parameters----------
#  -- basic loop extrusion parameters
SEPARATION = 200
LIFETIME = 300
N = 10000   # number of monomers
smcStepsPerBlock = 1  # now doing 1 SMC step per block 
steps = 250   # steps per block (now extrusion advances by one step per block)
stiff = 2
dens = 0.2
box = (N / dens) ** 0.33  # density = 0.1.
data = grow_cubic(N, int(box) - 2)  # creates a compact conformation 
block = 0  # starting block 

#folder 
folder = "test"

# new parameters because some things changed 
saveEveryBlocks = 10   # save every 10 blocks (saving every block is now too much almost)
skipSavedBlocksBeginning = 20  # how many blocks (saved) to skip after you restart LEF positions
totalSavedBlocks = 40  # how many blocks to save (number of blocks done is totalSavedBlocks * saveEveryBlocks)
restartMilkerEveryBlocks = 100 

# parameters for smc bonds 
smcBondWiggleDist = 0.2
smcBondDist = 0.5


# assertions for easy managing code below 
assert restartMilkerEveryBlocks % saveEveryBlocks == 0 
assert (skipSavedBlocksBeginning * saveEveryBlocks) % restartMilkerEveryBlocks == 0 
assert (totalSavedBlocks * saveEveryBlocks) % restartMilkerEveryBlocks == 0 

savesPerMilker = restartMilkerEveryBlocks // saveEveryBlocks
milkerInitsSkip = saveEveryBlocks * skipSavedBlocksBeginning  // restartMilkerEveryBlocks
milkerInitsTotal  = (totalSavedBlocks + skipSavedBlocksBeginning) * saveEveryBlocks // restartMilkerEveryBlocks
print("Milker will be initialized {0} times, first {1} will be skipped".format(milkerInitsTotal, milkerInitsSkip))

Milker will be initialized 6 times, first 2 will be skipped


## Bond updating functions (for increased speed)

In [3]:

class smcTranslocatorMilker(object):

    def __init__(self, smcTransObject):
        """
        :param smcTransObject: smc translocator object to work with
        """
        self.smcObject = smcTransObject
        self.allBonds = []

    def setParams(self, activeParamDict, inactiveParamDict):
        """
        A method to set parameters for bonds.
        It is a separate method because you may want to have a Simulation object already existing

        :param activeParamDict: a dict (argument:value) of addBond arguments for active bonds
        :param inactiveParamDict:  a dict (argument:value) of addBond arguments for inactive bonds

        """
        self.activeParamDict = activeParamDict
        self.inactiveParamDict = inactiveParamDict


    def setup(self, bondForce,  blocks = 100, smcStepsPerBlock = 1):
        """
        A method that milks smcTranslocator object
        and creates a set of unique bonds, etc.

        :param bondForce: a bondforce object (new after simulation restart!)
        :param blocks: number of blocks to precalculate
        :param smcStepsPerBlock: number of smcTranslocator steps per block
        :return:
        """


        if len(self.allBonds) != 0:
            raise ValueError("Not all bonds were used; {0} sets left".format(len(self.allBonds)))

        self.bondForce = bondForce

        #precalculating all bonds
        allBonds = []
        for dummy in range(blocks):
            self.smcObject.steps(smcStepsPerBlock)
            left, right = self.smcObject.getSMCs()
            bonds = [(int(i), int(j)) for i,j in zip(left, right)]
            allBonds.append(bonds)

        self.allBonds = allBonds
        self.uniqueBonds = list(set(sum(allBonds, [])))

        #adding forces and getting bond indices
        self.bondInds = []
        self.curBonds = allBonds.pop(0)

        for bond in self.uniqueBonds:
            paramset = self.activeParamDict if (bond in self.curBonds) else self.inactiveParamDict
            ind = bondForce.addBond(bond[0], bond[1], **paramset) # changed from addBond
            self.bondInds.append(ind)
        self.bondToInd = {i:j for i,j in zip(self.uniqueBonds, self.bondInds)}
        return self.curBonds,[]


    def step(self, context, verbose=False):
        """
        Update the bonds to the next step.
        It sets bonds for you automatically!
        :param context:  context
        :return: (current bonds, previous step bonds); just for reference
        """
        if len(self.allBonds) == 0:
            raise ValueError("No bonds left to run; you should restart simulation and run setup  again")

        pastBonds = self.curBonds
        self.curBonds = self.allBonds.pop(0)  # getting current bonds
        bondsRemove = [i for i in pastBonds if i not in self.curBonds]
        bondsAdd = [i for i in self.curBonds if i not in pastBonds]
        bondsStay = [i for i in pastBonds if i in self.curBonds]
        if verbose:
            print("{0} bonds stay, {1} new bonds, {2} bonds removed".format(len(bondsStay),
                                                                            len(bondsAdd), len(bondsRemove)))
        bondsToChange = bondsAdd + bondsRemove
        bondsIsAdd = [True] * len(bondsAdd) + [False] * len(bondsRemove)
        for bond, isAdd in zip(bondsToChange, bondsIsAdd):
            ind = self.bondToInd[bond]
            paramset = self.activeParamDict if isAdd else self.inactiveParamDict
            self.bondForce.setBondParameters(ind, bond[0], bond[1], **paramset)  # actually updating bonds
        self.bondForce.updateParametersInContext(context)  # now run this to update things in the context
        return self.curBonds, pastBonds
    
    
def initModel():
    # this jsut inits the simulation model. Put your previous init code here 
    birthArray = np.zeros(N, dtype=np.double) + 0.1
    deathArray = np.zeros(N, dtype=np.double) + 1. / LIFETIME
    stallDeathArray = np.zeros(N, dtype=np.double) + 1 / LIFETIME
    pauseArray = np.zeros(N, dtype=np.double)

    stallList = [1000, 3000, 5000, 7000, 9000]
    stallLeftArray = np.zeros(N, dtype = np.double)
    stallRightARray = np.zeros(N, dtype = np.double)
    for i in stallList:
        stallLeftArray[i] = 0.8
        stallRightARray[i] = 0.8

    smcNum = N // SEPARATION
    SMCTran = smcTranslocatorDirectional(birthArray, deathArray, stallLeftArray, stallRightARray, pauseArray,
                                         stallDeathArray, smcNum)
    return SMCTran


SMCTran = initModel()  # defining actual smc translocator object 


## Run simulation 

In [4]:

# ------------feed smcTran to the milker---
SMCTran.steps(1000000)  # first steps to "equilibrate" SMC dynamics. If desired of course. 


milker = smcTranslocatorMilker(SMCTran)   # now feed this thing to milker (do it once!)
#--------- end new code ------------


#!rm -r $folder
if os.path.exists(folder):
    shutil.rmtree(folder)
reporter = HDF5Reporter(folder=folder, max_data_length=100)



for milkerCount in range(milkerInitsTotal):
    doSave = milkerCount >= milkerInitsSkip
    
    # simulation parameters are defined below 
    a = Simulation(
            platform="cuda",
            integrator="variableLangevin", 
            error_tol=0.0002, 
            GPU = "0", 
            collision_rate=0.01, 
            N = len(data),
            reporters=[reporter],
            PBCbox=[box, box, box],
            precision="mixed")  # timestep not necessary for variableLangevin

    
    
    ############################## New code ##############################
    a.set_data(data)  # loads a polymer, puts a center of mass at zero
    
    
    # -----------Adding forces ---------------
    # sim.addForce(forces.sphericalConfinement(sim, density=0.3, k=1))
    #sim.addForce(forces.cylindricalConfinement(sim,cellRadius,bottom=-halfLength,top=halfLength,k=5))


    a.add_force(
        forcekits.polymer_chains(
            a,
            chains=[(0, None, None)],

                # By default the library assumes you have one polymer chain
                # If you want to make it a ring, or more than one chain, use self.setChains
                # self.setChains([(0,50,1),(50,None,0)]) will set a 50-monomer ring and a chain from monomer 50 to the end

            bond_force_func=forces.harmonic_bonds,
            bond_force_kwargs={
                'bondLength':1.0,
                'bondWiggleDistance':0.01, # Bond distance will fluctuate +- 0.05 on average
             },

            angle_force_func=forces.angle_force,
            angle_force_kwargs={
                'k':0.05
                # K is more or less arbitrary, k=4 corresponds to presistence length of 4,
                # k=1.5 is recommended to make polymer realistically flexible; k=8 is very stiff
            },

            nonbonded_force_func=forces.polynomial_repulsive,
            nonbonded_force_kwargs={
                'trunc':1.5, # this will let chains cross sometimes
                'radiusMult':1.05, # this is from old code
                #'trunc':10.0, # this will resolve chain crossings and will not let chain cross anymore
            },

            except_bonds=True,
             
        )
    )

    

#     ############################## OLD code ##############################
    a.step = block

    # ------------ initializing milker; adding bonds ---------
    # copied from addBond
    kbond = a.kbondScalingFactor / (smcBondWiggleDist ** 2)
    bondDist = smcBondDist * a.length_scale

    activeParams = {"length":bondDist,"k":kbond}
    inactiveParams = {"length":bondDist, "k":0}
    milker.setParams(activeParams, inactiveParams)
     
    # this step actually puts all bonds in and sets first bonds to be what they should be
    milker.setup(bondForce=a.force_dict['harmonic_bonds'],
                blocks=restartMilkerEveryBlocks,   # default value; milk for 100 blocks
                 smcStepsPerBlock=smcStepsPerBlock)  # now only one step of SMC per step
    print("Restarting milker")

    
    # If your simulation does not start, consider using energy minimization below
    a.local_energy_minimization() 
    
    
#     a.do_block(steps=steps,)#, increment=False)  # THIS RESULTS IN EK>10!!
    for i in range(restartMilkerEveryBlocks - 1):
        curBonds, pastBonds = milker.step(a.context)  # this updates bonds. You can do something with bonds here
        if i % saveEveryBlocks == (saveEveryBlocks - 2):  
            a.do_block(steps=steps)#, increment = doSave)    
            if doSave: 
#                 a.save()
                pickle.dump(curBonds, open(os.path.join(folder, "SMC{0}.dat".format(a.step)),'wb'))
        else:
            a.integrator.step(steps)  # do steps without getting the positions from the GPU (faster)

    data = a.get_data()  # save data and step, and delete the simulation
    block = a.step
    del a

    time.sleep(0.2)  # wait 200ms for sanity (to let garbage collector do its magic)
    
    
reporter.dump_data()

INFO:root:Performing local energy minimization
INFO:root:adding force harmonic_bonds 0
INFO:root:adding force angle 1
INFO:root:Using periodic boundary conditions
INFO:root:adding force polynomial_repulsive 2


Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 9999
Restarting milker


INFO:root:Particles loaded. Potential energy is 10.650488
INFO:root:before minimization eK=1.5239596041544532, eP=10.6504878145233, time=0.0 ps
INFO:root:after minimization eK=1.5239596041544532, eP=0.0440479921920967, time=0.0 ps
INFO:root:block    0 pos[1]=[14.3 14.4 14.7] dr=1.99 t=15.1ps kin=2.40 pot=1.39 Rg=12.091 dt=7.1fs dx=2.46pm 
INFO:root:block    1 pos[1]=[12.6 15.7 13.1] dr=2.83 t=32.9ps kin=2.76 pot=1.19 Rg=13.903 dt=7.1fs dx=2.63pm 
INFO:root:block    2 pos[1]=[13.3 16.9 14.7] dr=3.05 t=50.6ps kin=2.98 pot=1.11 Rg=15.566 dt=7.1fs dx=2.74pm 
INFO:root:block    3 pos[1]=[11.5 14.4 15.5] dr=3.03 t=68.4ps kin=3.08 pot=1.05 Rg=16.784 dt=7.1fs dx=2.78pm 
INFO:root:block    4 pos[1]=[12.9 13.4 14.3] dr=3.00 t=86.1ps kin=3.05 pot=1.05 Rg=17.550 dt=7.1fs dx=2.77pm 
INFO:root:block    5 pos[1]=[11.3 13.9 15.5] dr=2.96 t=104.5ps kin=3.02 pot=1.02 Rg=18.022 dt=7.7fs dx=2.98pm 
INFO:root:block    6 pos[1]=[11.5 13.3 15.7] dr=2.87 t=123.7ps kin=3.01 pot=1.01 Rg=18.204 dt=7.7fs dx=2.97p

Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 9999
Restarting milker


INFO:root:Particles loaded. Potential energy is 1.070524
INFO:root:before minimization eK=1.4943457692327853, eP=1.070523694738694, time=0.0 ps
INFO:root:after minimization eK=1.4943457692327853, eP=0.030612848807236456, time=0.0 ps
INFO:root:block    0 pos[1]=[12.2 9.2 11.3] dr=2.14 t=14.7ps kin=2.67 pot=1.28 Rg=17.973 dt=6.8fs dx=2.48pm 
INFO:root:block    1 pos[1]=[11.8 9.1 13.0] dr=2.71 t=31.7ps kin=2.76 pot=1.18 Rg=17.856 dt=6.8fs dx=2.53pm 
INFO:root:block    2 pos[1]=[11.5 10.5 14.2] dr=2.58 t=48.7ps kin=2.80 pot=1.15 Rg=17.740 dt=6.8fs dx=2.54pm 
INFO:root:block    3 pos[1]=[11.5 10.6 16.6] dr=2.61 t=66.2ps kin=2.87 pot=1.13 Rg=17.622 dt=7.4fs dx=2.79pm 
INFO:root:block    4 pos[1]=[10.4 9.9 14.2] dr=2.71 t=84.6ps kin=2.86 pot=1.10 Rg=17.541 dt=7.4fs dx=2.79pm 
INFO:root:block    5 pos[1]=[10.1 9.9 13.4] dr=2.61 t=103.1ps kin=2.84 pot=1.07 Rg=17.494 dt=7.4fs dx=2.78pm 
INFO:root:block    6 pos[1]=[13.5 10.2 15.3] dr=2.62 t=121.5ps kin=2.85 pot=1.06 Rg=17.418 dt=7.4fs dx=2.78pm 

...

INFO:root:Particles loaded. Potential energy is 1.039000
INFO:root:before minimization eK=1.5097342238648095, eP=1.0390003460425992, time=0.0 ps
INFO:root:after minimization eK=1.5097342238648095, eP=0.030139506143188003, time=0.0 ps
INFO:root:block    0 pos[1]=[8.8 11.6 14.1] dr=2.17 t=14.4ps kin=2.76 pot=1.35 Rg=17.328 dt=6.7fs dx=2.48pm 
INFO:root:block    1 pos[1]=[8.8 10.3 14.1] dr=2.59 t=31.2ps kin=3.00 pot=1.24 Rg=17.282 dt=6.7fs dx=2.58pm 
INFO:root:block    2 pos[1]=[8.9 9.9 15.0] dr=2.63 t=47.9ps kin=3.00 pot=1.20 Rg=17.250 dt=6.7fs dx=2.59pm 
INFO:root:block    3 pos[1]=[7.7 13.2 16.1] dr=2.69 t=65.8ps kin=2.97 pot=1.18 Rg=17.255 dt=7.3fs dx=2.79pm 
INFO:root:block    4 pos[1]=[3.1 16.4 14.4] dr=2.64 t=84.0ps kin=2.92 pot=1.15 Rg=17.268 dt=7.3fs dx=2.77pm 
INFO:root:block    5 pos[1]=[3.8 18.1 13.4] dr=2.63 t=102.1ps kin=2.90 pot=1.10 Rg=17.239 dt=7.3fs dx=2.76pm 
INFO:root:block    6 pos[1]=[1.8 14.8 10.0] dr=2.65 t=120.3ps kin=2.97 pot=1.05 Rg=17.236 dt=7.3fs dx=2.79pm 
IN

Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 9999
Restarting milker


INFO:root:Particles loaded. Potential energy is 1.090508
INFO:root:before minimization eK=1.502997898823275, eP=1.0905082711772003, time=0.0 ps
INFO:root:after minimization eK=1.502997898823275, eP=0.036438689946680404, time=0.0 ps
INFO:root:block    0 pos[1]=[16.8 5.0 2.7] dr=2.19 t=15.0ps kin=2.70 pot=1.20 Rg=17.273 dt=6.9fs dx=2.52pm 
INFO:root:block    1 pos[1]=[17.3 6.1 3.9] dr=2.60 t=32.2ps kin=2.73 pot=1.18 Rg=17.218 dt=6.9fs dx=2.54pm 
INFO:root:block    2 pos[1]=[18.2 6.9 2.9] dr=2.50 t=49.4ps kin=2.67 pot=1.13 Rg=17.205 dt=6.9fs dx=2.51pm 
INFO:root:block    3 pos[1]=[16.0 6.2 1.8] dr=2.49 t=67.4ps kin=2.58 pot=1.04 Rg=17.202 dt=7.4fs dx=2.66pm 
INFO:root:block    4 pos[1]=[16.4 6.3 4.4] dr=2.51 t=86.0ps kin=2.54 pot=1.02 Rg=17.220 dt=7.4fs dx=2.64pm 
INFO:root:block    5 pos[1]=[16.6 5.3 4.2] dr=2.47 t=104.5ps kin=2.51 pot=0.99 Rg=17.251 dt=7.4fs dx=2.63pm 
INFO:root:block    6 pos[1]=[16.5 5.6 5.8] dr=2.49 t=123.1ps kin=2.55 pot=0.96 Rg=17.305 dt=7.4fs dx=2.65pm 
INFO:root:

Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 9999
Restarting milker


INFO:root:Particles loaded. Potential energy is 0.960815
INFO:root:before minimization eK=1.497946180821967, eP=0.9608146569604147, time=0.0 ps
INFO:root:after minimization eK=1.497946180821967, eP=0.03659412678966886, time=0.0 ps
INFO:root:block    0 pos[1]=[12.3 7.2 6.2] dr=2.16 t=15.0ps kin=2.60 pot=1.13 Rg=17.433 dt=7.0fs dx=2.52pm 
INFO:root:block    1 pos[1]=[12.3 7.4 5.0] dr=2.55 t=32.5ps kin=2.74 pot=1.12 Rg=17.360 dt=7.0fs dx=2.59pm 
INFO:root:block    2 pos[1]=[13.4 6.0 5.2] dr=2.59 t=50.0ps kin=2.80 pot=1.09 Rg=17.333 dt=7.0fs dx=2.61pm 
INFO:root:block    3 pos[1]=[12.5 6.1 7.0] dr=2.60 t=67.5ps kin=2.89 pot=1.06 Rg=17.348 dt=7.0fs dx=2.66pm 
INFO:root:block    4 pos[1]=[12.9 6.0 7.1] dr=2.68 t=85.3ps kin=2.99 pot=1.07 Rg=17.362 dt=7.6fs dx=2.92pm 
INFO:root:block    5 pos[1]=[12.0 4.2 6.7] dr=2.77 t=104.2ps kin=2.91 pot=1.04 Rg=17.382 dt=7.6fs dx=2.89pm 
INFO:root:block    6 pos[1]=[12.7 5.0 5.7] dr=2.70 t=123.1ps kin=2.81 pot=1.01 Rg=17.435 dt=7.6fs dx=2.83pm 
INFO:root:b